In [1]:
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import ColumnTransformer
import string
from sklearn.linear_model  import LogisticRegression
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import pymorphy2

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baben_bakg1j1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
test = pd.read_csv("test.csv", index_col='ID')
train = pd.read_csv("train.csv", index_col='ID')
train.head(2)

,url,title,label
ID,,,
0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",0
1,www.kp.by,Эта песня стала известна многим телезрителям б...,0


In [6]:
test.head(5)

,url,title
ID,,
135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
135313,2gis.ru,67


In [228]:
print(test.isnull().sum())
print(train.isnull().sum())

url      0
title    0
dtype: int64
url      0
title    0
label    0
dtype: int64


In [244]:
y_train = train['label']
X_train = train.drop(['label'], axis=1)

**Удаление лишних символов + стоп слов + лемматизация**

In [245]:
def preprosessing_punctuation(text):
    return " ".join(text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).lower().split())

In [246]:
def preprosessing_without_lem(text, stop_words, morph):
    words = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).lower().split()
    new_words = []
    for word in words:
        if word not in stop_words:
            new_words.append(word)
    return " ".join(new_words)

In [247]:
def preprosessing(text, stop_words, morph):
    words = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).lower().split()
    lemmatized_words = []
    for word in words:
        if word not in stop_words:
            lemmatized_word = morph.parse(word)[0].normal_form
            lemmatized_words.append(lemmatized_word)
    return " ".join(lemmatized_words)
    

**По итогам многих экспериментов выяснилось, что изначально лучше всего просто избавиться от лишних символов и привести все к нижнему регистру. Удаление стоп слов и лемматизация только ухудшали результат, а url лучше вообще не торогать**

In [248]:
stop_words = set(stopwords.words("russian")) | set(stopwords.words("english"))
morph = pymorphy2.MorphAnalyzer()

#X_train['url'] = X_train['url'].apply(lambda x: preprosessing_without_lem(x, stop_words, morph))
#X_train['title'] = X_train['title'].apply(lambda x: preprosessing(x, stop_words, morph))
#test['title'] = test['title'].apply(lambda x: preprosessing(x, stop_words, morph))
#test['url'] = test['url'].apply(lambda x: preprosessing(x, stop_words, morph))

#X_train['url'] = X_train['url'].apply(lambda x: preprosessing_punctuation(x))
X_train['title'] = X_train['title'].apply(lambda x: preprosessing_punctuation(x))
test['title'] = test['title'].apply(lambda x: preprosessing_punctuation(x))
#test['url'] = test['url'].apply(lambda x: preprosessing_punctuation(x))


**Использование отдельных vectorizer-ов для url и title**

In [249]:
preprocessor = ColumnTransformer(
    transformers=[
        ('url', TfidfVectorizer(
            analyzer="char_wb", ngram_range=(2, 4),
            min_df=3, max_df=0.75), 'url'),
        ('title', CountVectorizer(
            stop_words=stop_words, min_df=3,
            max_df=0.75), 'title')
    ])

In [250]:
X_train_preprocessed = preprocessor.fit_transform(X_train)
test_preprocessed = preprocessor.transform(test)

**Логистическая регрессия с лучшими подобранными гиперпараметрами (Были протестированы и другие алогоритмы, но они показали себя хуже)**

In [251]:
model = LogisticRegression(
    C=1.5, solver='lbfgs', 
    class_weight='balanced', max_iter=1500)

model.fit(X_train_preprocessed, y_train)
y_pred = model.predict(test_preprocessed)

**Реализую гипотезу об отнесении сайта к положительному классу, при вхождении в его title и url 'плохих слов'
(Получился небольшой прирост точности)**

In [252]:
bad_words = []
with open('words.txt', 'r', encoding='utf-8') as file:
    for line in file:
        bad_words.append(line.strip())
        
for i, url in enumerate(test['url']):
    for word in bad_words:
        if word in url:
            y_pred[i] = 1
            
for i, title in enumerate(test['title']):
    for word in bad_words:
        if word in title:
            y_pred[i] = 1

In [253]:
submission = pd.DataFrame({'ID':test.reset_index()['ID'], 'label':y_pred})
submission.to_csv('submission.csv', index=False)
submission.head()

,ID,label
0,135309,0
1,135310,0
2,135311,0
3,135312,1
4,135313,0
